In [52]:
import tensorflow as tf
import numpy as np

In [53]:
#mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/",one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [54]:
tf.reset_default_graph()
n_input = 28  # MNIST输入数据
n_step = 28   # 序列个数
n_hidden = 128
n_classes = 10

In [55]:
x = tf.placeholder("float",[None,n_step,n_input])
y = tf.placeholder("float",[None,n_classes])

x1 = tf.unstack(x,n_step,1)

In [56]:
#basic LSTMCell
#lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden,forget_bias=1.0)
#outputs,stas = tf.contrib.rnn.static_rnn(lstm_cell,x1,dtype=tf.float32)   #0.95

#LSTMCell
#lstm_cell = tf.contrib.rnn.LSTMCell(n_hidden,forget_bias=1.0)
#outputs,stas = tf.contrib.rnn.static_rnn(lstm_cell,x1,dtype=tf.float32)   #0.97

#gru
#gru = tf.contrib.rnn.GRUCell(n_hidden)
#outputs,stas = tf.contrib.rnn.static_rnn(gru,x1,dtype=tf.float32)          #0.97

#动态RNN
gru = tf.contrib.rnn.GRUCell(n_hidden)
outputs,_  = tf.nn.dynamic_rnn(gru,x,dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])                                  #0.99

In [57]:
pred = tf.contrib.layers.fully_connected(outputs[-1],n_classes,activation_fn=None)

In [58]:
#learning param
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10
#Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [59]:
# 启动session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_step, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # 计算批次数据的准确率
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print (" Finished!")

    # 计算准确率 for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_step, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 2.146743, Training Accuracy= 0.26562
Iter 2560, Minibatch Loss= 1.966783, Training Accuracy= 0.27344
Iter 3840, Minibatch Loss= 1.661038, Training Accuracy= 0.42969
Iter 5120, Minibatch Loss= 1.309070, Training Accuracy= 0.51562
Iter 6400, Minibatch Loss= 1.302644, Training Accuracy= 0.58594
Iter 7680, Minibatch Loss= 1.109557, Training Accuracy= 0.58594
Iter 8960, Minibatch Loss= 0.900027, Training Accuracy= 0.73438
Iter 10240, Minibatch Loss= 0.806835, Training Accuracy= 0.71094
Iter 11520, Minibatch Loss= 0.674669, Training Accuracy= 0.78906
Iter 12800, Minibatch Loss= 0.688387, Training Accuracy= 0.75781
Iter 14080, Minibatch Loss= 0.492256, Training Accuracy= 0.83594
Iter 15360, Minibatch Loss= 0.766805, Training Accuracy= 0.77344
Iter 16640, Minibatch Loss= 0.502821, Training Accuracy= 0.87500
Iter 17920, Minibatch Loss= 0.364338, Training Accuracy= 0.87500
Iter 19200, Minibatch Loss= 0.509132, Training Accuracy= 0.85156
Iter 20480, Minibatch Loss= 0.31